In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
data = pd.read_csv('target_dataset.csv')

data['Tweet'] = data['Tweet'].str.replace(r'http\S+', '', case=False)
data['Tweet'] = data['Tweet'].str.replace(r'@\S+', '', case=False)

analyzer = SentimentIntensityAnalyzer()

sentiments = []
for tweet in data['Tweet']:
    scores = analyzer.polarity_scores(tweet)
    if scores['compound'] > 0:
        sentiment = '1'
    elif scores['compound'] < 0:
        sentiment = '-1'
    else:
        sentiment = '0'
    sentiments.append(sentiment)

data['sentiment'] = sentiments

print(data['sentiment'].value_counts())

<ipython-input-7-d6ecde9fa0bc>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Tweet'] = data['Tweet'].str.replace(r'http\S+', '', case=False)
<ipython-input-7-d6ecde9fa0bc>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Tweet'] = data['Tweet'].str.replace(r'@\S+', '', case=False)


-1    39523
1      9167
0      1310
Name: sentiment, dtype: int64


In [ ]:
data.head()

,Date,User,Tweet,Retweets,Followers,Likes,char_counts,word_counts,avg_wordlength,stopwords_counts,hashtag_counts,mentions_counts,digits_counts,uppercase_counts,Target,sentiment
0,2023-02-08 05:06:05+00:00,KenRaty,wallybigwall joneshospodtx cfidd i love it but...,0,106,0,158,31,5.096774,18,0,0,0,0,1,-1
1,2023-02-08 05:06:01+00:00,DileepKumarPak,search and rescue operations resumed at the si...,0,280,0,127,19,6.684211,4,0,0,1,0,1,-1
2,2023-02-08 05:06:00+00:00,DSWDfo7,our community volunteers in brgy canagahan of ...,0,1904,0,176,29,6.068966,7,0,0,2,0,1,-1
3,2023-02-08 05:05:48+00:00,migzu17,yugivee by accident i was originally going to ...,0,416,0,153,38,4.026316,23,0,0,0,0,1,1
4,2023-02-08 05:05:47+00:00,darrellriffic,hishyar09205113 mdoe99500351 bozturkserkan dor...,0,54,0,286,58,4.931034,36,0,0,3,0,1,1


In [ ]:
data.to_csv('twitter_sentiment_dataset.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def ranking():
  # Load dataset
  data = pd.read_csv('twitter_sentiment_dataset.csv')

  # Preprocess tweets (e.g. remove stop words, punctuation, etc.)
  # ...

  # Convert preprocessed tweets into feature vectors using TF-IDF
  tfidf_vectorizer = TfidfVectorizer()
  tfidf_matrix = tfidf_vectorizer.fit_transform(data['Tweet'])

  # Compute cosine similarity matrix
  cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

  # Rank tweets based on cosine similarity to a query tweet
  query_tweet_index = 10  # index of the query tweet in the dataset
  cosine_sim_scores = list(enumerate(cosine_sim[query_tweet_index]))
  cosine_sim_scores_sorted = sorted(cosine_sim_scores, key=lambda x: x[1], reverse=True)
  top_tweets_indices = [i for i, _ in cosine_sim_scores_sorted[1:11]]  # top 10 most similar tweets (excluding query tweet)
  top_tweets = data.iloc[top_tweets_indices]['Tweet']

  # Print top 10 most similar tweets
  for i, tweet in enumerate(top_tweets):
      print(f"Rank {i+1}: {tweet}")